# Kiva collaborative filtering
## Polara framework

***
### Imports

In [1]:
# essentials
import os
import sys
import csv
import itertools
import copy
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from scipy.sparse import csr_matrix, lil_matrix

# polara framework
from polara.recommender.data import RecommenderData
from polara.recommender.models import SVDModel, PopularityModel, RandomModel, CooccurrenceModel
from polara.recommender.external.implicit.ialswrapper import ImplicitALS
from polara.datasets.movielens import get_movielens_data
from polara.evaluation import evaluation_engine as ee
from polara.evaluation.plotting import show_hit_rates, show_precision_recall, show_ranking, show_relevance

# utilities
import codecs
import logging
import time
import tqdm

# visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt

# serialization
import pickle

In [2]:
pd.set_option('display.max_columns', 40)

In [3]:
%env MKL_NUM_THREADS=1
logging.basicConfig(level=logging.DEBUG)

env: MKL_NUM_THREADS=1


***
### Pickle loads

In [ ]:
loans_table = pickle.load(open("pickle/loans_table.p", "rb"))
funded_loans_table = pickle.load(open("pickle/funded_loans_table.p", "rb"))
funded_loan_ids_set = pickle.load(open("pickle/funded_loan_ids_set.p", "rb"))

In [ ]:
utility_matrix = pickle.load(open("pickle/utility_matrix.p", "rb"))

In [ ]:
utility_matrix_df = pickle.load(open("pickle/utility_matrix_df.p", "rb"))

In [ ]:
utility_matrix_df.head()

***
### Dataset reading

In [ ]:
loans_table = pd.read_csv('additional-kiva-snapshot/loans.csv')
loans_table = loans_table.sort_values(by='raised_time')

In [ ]:
funded_loans_table = loans_table[loans_table.status == 'funded']

In [ ]:
START_DATE = '2013-10-01'
END_DATE = '2015-05-01'

mask = (funded_loans_table['raised_time'] > START_DATE) & (funded_loans_table['raised_time'] <= END_DATE)
funded_loans_table = funded_loans_table.loc[mask]
del mask

funded_loan_ids_set = set(funded_loans_table['loan_id'])

In [ ]:
# pickle.dump(loans_table, open("pickle/loans_table.p", "wb"))
# pickle.dump(funded_loans_table, open("pickle/funded_loans_table.p", "wb"))
# pickle.dump(funded_loan_ids_set, open("pickle/funded_loan_ids_set.p", "wb"))

In [ ]:
# free ram
del loans_table
del funded_loans_table

In [ ]:
loans = set()
lenders = set()
loans_lenders_dict = {}

with open('additional-kiva-snapshot/loans_lenders.csv', newline='', encoding="utf8") as csvfile:
    csv_reader = csv.reader(csvfile)
    line_num = 0
    for row in csv_reader:
        if line_num == 0:
            line_num += 1
            continue
        loan_id, lender_ids = row
        loan_id = int(loan_id)
        if loan_id not in funded_loan_ids_set:
            continue
        
        loans.add(loan_id)
        new_lenders = set(lender_ids.split(", "))
        loans_lenders_dict[loan_id] = new_lenders
        lenders.update(new_lenders)
        line_num += 1

loans = list(loans)
lenders = list(lenders)

print('Loans-lenders dict filled')

***
### Utility matrix creation

In [ ]:
lenders_reverse_index = {k: v for v, k in enumerate(lenders)}
utility_matrix = lil_matrix((len(loans), len(lenders)), dtype=np.float64)

In [ ]:
for loan_index, loan in enumerate(loans):
    for lender in loans_lenders_dict[loan]:
        lender_index = lenders_reverse_index[lender]
        utility_matrix[loan_index, lender_index] = 1.0

print('Filled utility matrix')

In [ ]:
utility_matrix = utility_matrix.tocsr()

In [ ]:
# pickle.dump(utility_matrix, open("pickle/utility_matrix.p", "wb"))

#### Conversion to SparseDataFrame

In [ ]:
utility_matrix_df = pd.SparseDataFrame(utility_matrix)

In [ ]:
# pickle.dump(utility_matrix_df, open("pickle/utility_matrix_df.p", "wb"))

In [ ]:
def write_matrix_to_csv(matrix, filename):
    if not isinstance(matrix, scipy.sparse.coo_matrix):
        matrix = matrix.tocoo()
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("%s,%s,%s\n" % ('itemid', 'userid', 'feedback'))
        for i,j,v in zip(matrix.row, matrix.col, matrix.data):
            f.write("%s,%s,%s\n" % (i, j, v))

#### Load Kiva dataframe

In [4]:
kiva_dataframe = pd.read_csv('kiva_dataframe.csv', engine='c')

In [5]:
kiva_dataframe.shape

(5339455, 3)

In [6]:
kiva_dataframe.head()

,itemid,userid,feedback
0,0,7703,1.0
1,0,8354,1.0
2,0,9000,1.0
3,0,10247,1.0
4,0,21284,1.0


***
### Polara example

In [ ]:
# define models
ml_data = get_movielens_data(get_genres=False)
data_model = RecommenderData(ml_data, 'userid', 'movieid', 'rating')

# data_model = RecommenderData(kiva_dataframe, 'userid', 'itemid', 'feedback')

svd = SVDModel(data_model)
popular = PopularityModel(data_model)
random = RandomModel(data_model)
models = [svd, popular, random]

metrics = ['ranking', 'relevance'] # metrics for evaluation: NDGC, Precision, Recall, etc.
folds = [1, 2, 3, 4, 5] # use all 5 folds for cross-validation (default)
topk_values = [1, 5, 10, 20, 50] # values of k to experiment with

# run 5-fold CV experiment
result = ee.run_cv_experiment(models, folds, metrics,
                              fold_experiment=ee.topk_test,
                              topk_list=topk_values)

In [ ]:
# calculate average values across all folds for e.g. relevance metrics
scores = result.mean(axis=0, level=['top-n', 'model']) # use .std instead of .mean for standard deviation
# scores.xs('nDCG', level='metric', axis=1).unstack('model')

In [ ]:
scores

***
### Cross-validation

In [7]:
data_model = RecommenderData(kiva_dataframe, 'userid', 'itemid', 'feedback', seed=0)
data_model.warm_start = False

start = time.time()
data_model.prepare()
logging.debug("Prepared data in %0.2fs", time.time() - start)

Preparing data...
207 unique itemid's within 228 holdout interactions were filtered. Reason: not in the training data.
85452 unique userid's within 136096 holdout interactions were filtered. Reason: not in the training data.
114 of 44225 userid's were filtered out from holdout. Reason: incompatible number of items.
Done.
There are 5070574 events in the training and 132333 events in the holdout.


DEBUG:root:Prepared data in 132.61s


In [8]:
random = RandomModel(data_model)
popular = PopularityModel(data_model)
svd = SVDModel(data_model)

Generate a list of ALS models by parameter grid:

In [9]:
def get_base_model():
    model = ImplicitALS(data_model)
    model.epsilon = 1e-8
    model.weight_func = np.log2
    model.use_gpu = False
    return model

def get_grid_models(cv_param_grid):
    models = []
    keys, values = zip(*cv_param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        next_model = get_base_model()
        name = 'ALS'
        for k, v in params.items():
            setattr(next_model, k, v)
            name += '_%s-%s' % (k, v)
        
        next_model.method = name
        models.append(next_model)
    return models

Cross-validation setup:

In [10]:
cv_param_grid = {
    'rank': [10, 50, 100],
    'regularization': [0.01, 0.1, 1],
    'alpha': [1, 50, 100],
    'num_epochs': [15, 30, 40]
}

In [11]:
basic_models = [random, popular, svd]
als_models = get_grid_models(cv_param_grid)
models = basic_models + als_models

metrics = ['ranking', 'relevance'] # metrics for evaluation: NDGC, Precision, Recall, etc.
folds = [1, 2, 3]
topk_values = [5, 10, 20] # values of k to experiment with

In [15]:
start = time.time()

# run 5-fold CV experiment
result = ee.run_cv_experiment(models, folds, metrics,
                              fold_experiment=ee.topk_test,
                              topk_list=topk_values)


logging.debug("Cross-validation experiment finished in %0.2fs", time.time() - start)

Preparing data...
211 unique itemid's within 234 holdout interactions were filtered. Reason: not in the training data.
85443 unique userid's within 136172 holdout interactions were filtered. Reason: not in the training data.
100 of 44237 userid's were filtered out from holdout. Reason: incompatible number of items.
Done.
There are 5070446 events in the training and 132411 events in the holdout.
PureSVD training time: 3.353s


DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.172s
DEBUG:implicit:Calculated transpose in 0.127s
DEBUG:implicit:Initialized factors in 0.14058566093444824
DEBUG:implicit:Running 15 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:21<00:00,  1.43s/it]


ALS_rank-10_regularization-0.01_alpha-1_num_epochs-15 training time: 21.566s


DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.187s
DEBUG:implicit:Calculated transpose in 0.125s
DEBUG:implicit:Initialized factors in 0.14058518409729004
DEBUG:implicit:Running 30 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:42<00:00,  1.26s/it]


ALS_rank-10_regularization-0.01_alpha-1_num_epochs-30 training time: 43.174s


...

DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.172s
DEBUG:implicit:Calculated transpose in 0.141s
DEBUG:implicit:Initialized factors in 1.2965657711029053
DEBUG:implicit:Running 30 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [01:17<00:00,  2.37s/it]


ALS_rank-100_regularization-1_alpha-100_num_epochs-30 training time: 01m:19s


DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.172s
DEBUG:implicit:Calculated transpose in 0.141s
DEBUG:implicit:Initialized factors in 1.2653229236602783
DEBUG:implicit:Running 40 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 40.0/40 [01:41<00:00,  2.35s/it]


ALS_rank-100_regularization-1_alpha-100_num_epochs-40 training time: 01m:43s
Evaluated model in 256.29s
Evaluated model in 106.83s
Evaluated model in 164.28s
Evaluated model in 128.53s
Evaluated model in 128.05s
Evaluated model in 128.52s
Evaluated model in 123.78s
Evaluated model in 123.38s
Evaluated model in 122.93s
Evaluated model in 129.44s
Evaluated model in 125.15s
Evaluated model in 127.06s
Evaluated model in 129.44s
Evaluated model in 129.61s
Evaluated model in 129.29s
Evaluated model in 127.48s
Evaluated model in 126.75s
Evaluated model in 127.46s
Evaluated model in 131.67s
Evaluated model in 128.33s
Evaluated model in 127.24s
Evaluated model in 130.14s
Evaluated model in 132.58s
Evaluated model in 130.33s
Evaluated model in 125.96s
Evaluated model in 124.35s
Evaluated model in 125.83s
Evaluated model in 130.47s
Evaluated model in 126.31s
Evaluated model in 125.90s
Evaluated model in 147.72s
Evaluated model in 146.75s
Evaluated model in 146.31s
Evaluated model in 143.49s
Evalu

DEBUG:root:Cross-validation experiment finished in 51275.64s


In [16]:
pickle.dump(result, open("pickle/result.p", "wb"))

In [ ]:
# holdout_sizes = [1, 2, 5]

# result = ee.run_cv_experiment(models, folds, metrics,
#                               fold_experiment=ee.holdout_test,
#                               holdout_sizes=holdout_sizes)

In [17]:
# calculate average values across all folds for e.g. relevance metrics
scores = result.mean(axis=0, level=['top-n', 'model']) # use .std instead of .mean for standard deviation
# scores.xs('recall', level='metric', axis=1).unstack('model')
scores

type                                                     relevance            \
metric                                                   precision    recall   
top-n model                                                                    
5     RND                                                 0.000012  0.000020   
      MP                                                  0.001156  0.001927   
      PureSVD                                             0.001158  0.001929   
      ALS_rank-10_regularization-0.01_alpha-1_num_epo...  0.000909  0.001516   
      ALS_rank-10_regularization-0.01_alpha-1_num_epo...  0.000958  0.001597   
      ALS_rank-10_regularization-0.01_alpha-1_num_epo...  0.000937  0.001561   
      ALS_rank-10_regularization-0.01_alpha-50_num_ep...  0.000708  0.001180   
      ALS_rank-10_regularization-0.01_alpha-50_num_ep...  0.000726  0.001211   
      ALS_rank-10_regularization-0.01_alpha-50_num_ep...  0.000808  0.001347   
      ALS_rank-10_regularization-0.01_alpha-100_num_e...  0.000605  0.001009   
      ALS_rank-10_regularization-0.01_alpha-100_num_e...  0.000530  0.000883   
      ALS_rank-10_regularization-0.01_alpha-100_num_e...  0.000549  0.000915   
      ALS_rank-10_regularization-0.1_alpha-1_num_epoc...  0.000917  0.001528   
      ALS_rank-10_regularization-0.1_alpha-1_num_epoc...  0.001008  0.001680   
      ALS_rank-10_regularization-0.1_alpha-1_num_epoc...  0.000906  0.001511   
      ALS_rank-10_regularization-0.1_alpha-50_num_epo...  0.000697  0.001162   
      ALS_rank-10_regularization-0.1_alpha-50_num_epo...  0.000769  0.001281   
      ALS_rank-10_regularization-0.1_alpha-50_num_epo...  0.000755  0.001258   
      ALS_rank-10_regularization-0.1_alpha-100_num_ep...  0.000530  0.000883   
      ALS_rank-10_regularization-0.1_alpha-100_num_ep...  0.000572  0.000953   
      ALS_rank-10_regularization-0.1_alpha-100_num_ep...  0.000587  0.000978   
      ALS_rank-10_regularization-1_alpha-1_num_epochs-15  0.000917  0.001528   
      ALS_rank-10_regularization-1_alpha-1_num_epochs-30  0.000947  0.001579   
      ALS_rank-10_regularization-1_alpha-1_num_epochs-40  0.000987  0.001644   
      ALS_rank-10_regularization-1_alpha-50_num_epoch...  0.000666  0.001110   
      ALS_rank-10_regularization-1_alpha-50_num_epoch...  0.000813  0.001354   
      ALS_rank-10_regularization-1_alpha-50_num_epoch...  0.000722  0.001203   
      ALS_rank-10_regularization-1_alpha-100_num_epoc...  0.000604  0.001006   
      ALS_rank-10_regularization-1_alpha-100_num_epoc...  0.000581  0.000968   
      ALS_rank-10_regularization-1_alpha-100_num_epoc...  0.000561  0.000935   
...                                                            ...       ...   
20    ALS_rank-50_regularization-1_alpha-100_num_epoc...  0.000983  0.006551   
      ALS_rank-50_regularization-1_alpha-100_num_epoc...  0.001252  0.008350   
      ALS_rank-50_regularization-1_alpha-100_num_epoc...  0.001341  0.008937   
      ALS_rank-100_regularization-0.01_alpha-1_num_ep...  0.001833  0.012220   
      ALS_rank-100_regularization-0.01_alpha-1_num_ep...  0.001903  0.012687   
      ALS_rank-100_regularization-0.01_alpha-1_num_ep...  0.001900  0.012664   
      ALS_rank-100_regularization-0.01_alpha-50_num_e...  0.001915  0.012767   
      ALS_rank-100_regularization-0.01_alpha-50_num_e...  0.002172  0.014482   
      ALS_rank-100_regularization-0.01_alpha-50_num_e...  0.002229  0.014863   
      ALS_rank-100_regularization-0.01_alpha-100_num_...  0.001637  0.010914   
      ALS_rank-100_regularization-0.01_alpha-100_num_...  0.001981  0.013206   
      ALS_rank-100_regularization-0.01_alpha-100_num_...  0.002073  0.013819   
      ALS_rank-100_regularization-0.1_alpha-1_num_epo...  0.001852  0.012349   
      ALS_rank-100_regularization-0.1_alpha-1_num_epo...  0.001881  0.012538   
      ALS_rank-100_regularization-0.1_alpha-1_num_epo...  0.001920  0.012797   
      ALS_rank-100_regularization-0.1_alpha-50_num_ep...  0.001867  0.012444   


In [18]:
result

type                                                          relevance  \
metric                                                        precision   
fold top-n model                                                          
1    5     RND                                                 0.000005   
           MP                                                  0.001115   
           PureSVD                                             0.001355   
           ALS_rank-10_regularization-0.01_alpha-1_num_epo...  0.000834   
           ALS_rank-10_regularization-0.01_alpha-1_num_epo...  0.000875   
           ALS_rank-10_regularization-0.01_alpha-1_num_epo...  0.000893   
           ALS_rank-10_regularization-0.01_alpha-50_num_ep...  0.000634   
           ALS_rank-10_regularization-0.01_alpha-50_num_ep...  0.000662   
           ALS_rank-10_regularization-0.01_alpha-50_num_ep...  0.000798   
           ALS_rank-10_regularization-0.01_alpha-100_num_e...  0.000598   
           ALS_rank-10_regularization-0.01_alpha-100_num_e...  0.000557   
           ALS_rank-10_regularization-0.01_alpha-100_num_e...  0.000530   
           ALS_rank-10_regularization-0.1_alpha-1_num_epoc...  0.000829   
           ALS_rank-10_regularization-0.1_alpha-1_num_epoc...  0.000947   
           ALS_rank-10_regularization-0.1_alpha-1_num_epoc...  0.000938   
           ALS_rank-10_regularization-0.1_alpha-50_num_epo...  0.000684   
           ALS_rank-10_regularization-0.1_alpha-50_num_epo...  0.000766   
           ALS_rank-10_regularization-0.1_alpha-50_num_epo...  0.000798   
           ALS_rank-10_regularization-0.1_alpha-100_num_ep...  0.000485   
           ALS_rank-10_regularization-0.1_alpha-100_num_ep...  0.000494   
           ALS_rank-10_regularization-0.1_alpha-100_num_ep...  0.000548   
           ALS_rank-10_regularization-1_alpha-1_num_epochs-15  0.000902   
           ALS_rank-10_regularization-1_alpha-1_num_epochs-30  0.000843   
           ALS_rank-10_regularization-1_alpha-1_num_epochs-40  0.000865   
           ALS_rank-10_regularization-1_alpha-50_num_epoch...  0.000603   
           ALS_rank-10_regularization-1_alpha-50_num_epoch...  0.000852   
           ALS_rank-10_regularization-1_alpha-50_num_epoch...  0.000657   
           ALS_rank-10_regularization-1_alpha-100_num_epoc...  0.000535   
           ALS_rank-10_regularization-1_alpha-100_num_epoc...  0.000544   
           ALS_rank-10_regularization-1_alpha-100_num_epoc...  0.000603   
...                                                                 ...   
3    20    ALS_rank-50_regularization-1_alpha-100_num_epoc...  0.000998   
           ALS_rank-50_regularization-1_alpha-100_num_epoc...  0.001263   
           ALS_rank-50_regularization-1_alpha-100_num_epoc...  0.001384   
           ALS_rank-100_regularization-0.01_alpha-1_num_ep...  0.001827   
           ALS_rank-100_regularization-0.01_alpha-1_num_ep...  0.001878   
           ALS_rank-100_regularization-0.01_alpha-1_num_ep...  0.001916   
           ALS_rank-100_regularization-0.01_alpha-50_num_e...  0.001944   
           ALS_rank-100_regularization-0.01_alpha-50_num_e...  0.002188   
           ALS_rank-100_regularization-0.01_alpha-50_num_e...  0.002261   
           ALS_rank-100_regularization-0.01_alpha-100_num_...  0.001661   
           ALS_rank-100_regularization-0.01_alpha-100_num_...  0.002027   
           ALS_rank-100_regularization-0.01_alpha-100_num_...  0.002115   
           ALS_rank-100_regularization-0.1_alpha-1_num_epo...  0.001837   
           ALS_rank-100_regularization-0.1_alpha-1_num_epo...  0.001861   
           ALS_rank-100_regularization-0.1_alpha-1_num_epo...  0.001938   
           ALS_rank-100_regularization-0.1_alpha-50_num_ep...  0.001956   
           ALS_rank-100_regularization-0.1_alpha-50_num_ep...  0.002186   
           ALS_rank-100_regularization-0.1_alpha-50_num_ep...  0.002281   
           ALS_rank-100_regularization-0.1_alpha-100_num_e...  0.001576   
           ALS_rank-100_r

***
### Variable sizes

In [14]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

               RecommenderData:   2.0KiB
                   ImplicitALS:   2.0KiB
                      SVDModel:   1.4KiB
               PopularityModel:   1.4KiB
                   RandomModel:   1.4KiB
                    csr_matrix:   1.0KiB
                    lil_matrix:   1.0KiB
             CooccurrenceModel:   1.0KiB
                           _i1:   917.0B
                    als_models:   768.0B


In [13]:
del kiva_dataframe